## Part 1: Explore the Data

Import the data and use Pandas to learn more about the dataset.

In [ ]:
import pandas as pd

df = pd.read_csv('../Resources/client_dataset.csv')
df.head()

In [ ]:
# View the column names in the data
column_names = df.columns
print(column_names)

In [ ]:
# Use the describe function to gather some basic statistics
statistics = df.describe()
print(statistics)

In [ ]:
# Use this space to do any additional research
# and familiarize yourself with the data.
df.info()


In [ ]:
# What three item categories had the most entries?
value_counts = df['category'].value_counts()
print(value_counts)

In [ ]:
# For the category with the most entries, which subcategory had the most entries?
most_common_category = df["category"].value_counts().idxmax()
filtered_df = df[df["category"] == most_common_category]
most_common_subcategory = filtered_df["subcategory"].value_counts().idxmax()
print(most_common_subcategory)


In [ ]:
# Which five clients had the most entries in the data?
client_counts = df['client_id'].value_counts()
clients_sorted = client_counts.sort_values(ascending=False)
top_five_clients = clients_sorted.head(5)
print(top_five_clients)

In [ ]:
# Store the client ids of those top 5 clients in a list.
top_five_client_ids = top_five_clients.index.tolist()
print(top_five_client_ids)

In [ ]:
# How many total units (the qty column) did the client with the most entries order order?
client_total_units = df.groupby('client_id')['qty'].sum()
most_common_client_id = client_total_units.idxmax()
total_units_most_most_common_client = client_total_units.loc[most_common_client_id]
print(total_units_most_most_common_client)

## Part 2: Transform the Data
Do we know that this client spent the more money than client 66037? If not, how would we find out? Transform the data using the steps below to prepare it for analysis.

In [ ]:
# Create a column that calculates the subtotal for each line using the unit_price and the qty
df["subtotal"] = df["unit_price"] * df["qty"]
print(df)

In [ ]:
# Create a column for shipping price.
# Assume a shipping price of $7 per pound for orders over 50 pounds and $10 per pound for items 50 pounds or under.
def calculate_shipping_price(unit_weight):
    if unit_weight >50:
        return 7 * unit_weight
    else:
        return 10 * unit_weight

df["shipping_price"] = df["unit_weight"].apply(calculate_shipping_price)
print(df)

In [ ]:
# Create a column for the total price using the subtotal and the shipping price along with a sales tax of 9.25%
sales_tax_rate = 0.0925
df["sales_tax"] = (df["subtotal"]+ df["shipping_price"]) * sales_tax_rate
df["total_price"] = df["subtotal"] + df["shipping_price"] +df["sales_tax"]
print(df)


In [ ]:
# Create a column for the cost of each line using unit cost, qty, and
# shipping price (assume the shipping cost is exactly what is charged to the client).
df["line_cost"] = df["unit_cost"] * df["qty"] + df["shipping_price"]
print(df)



In [ ]:
# Create a column for the profit of each line using line cost and line price

df["profit"] = df["total_price"] - df["line_cost"]
print(df)

## Part 3: Confirm your work
You have email receipts showing that the total prices for 3 orders. Confirm that your calculations match the receipts. Remember, each order has multiple lines.

Order ID 2742071 had a total price of \$152,811.89

Order ID 2173913 had a total price of \$162,388.71

Order ID 6128929 had a total price of \$923,441.25


In [ ]:
# Check your work using the totals above
order_id_to_print = "2742071"
filtered_df = df[df["order_id"] == order_id_to_print]
print(order_id_to_print)
for index, row in filtered_df.iterrows():
    print("Line {}: Total_Price:".format (index + 1,
        row["total_price"]   
    ))
                                           


## Part 4: Summarize and Analyze
Use the new columns with confirmed values to find the following information.

In [28]:
# How much did each of the top 5 clients by quantity spend? Check your work from Part 1 for client ids.
data = {
    "client": ["33615", "66037", "46820", "38378", "24741"],
    "qty": ["220", "211", "209", "207", "207"]
}
df = pd.DataFrame(data)

client_spend = {}
for index, row in df.iterrows():
    client = row["client"]
    quantity = row["qty"]
    if client in client_spend:
        client_spend[client] += quantity
    else:
        client_spend[client] = quantity

sorted_clients = sorted(client_spend.items(), key=lambda x: x[1], reverse=True)

top_5_clients = sorted_clients[:5]

for client, quantity in top_5_clients:
    print(f"Client {client} spent a total quantity of {quantity}")

Client 33615 spent a total quantity of 220
Client 66037 spent a total quantity of 211
Client 46820 spent a total quantity of 209
Client 38378 spent a total quantity of 207
Client 24741 spent a total quantity of 207


In [ ]:
# Create a summary DataFrame showing the totals for the for the top 5 clients with the following information:
# total units purchased, total shipping price, total revenue, and total profit. 
data = {
    "client": ["33615", "66037", "46820", "38378", "24741"],
    "qty": [10, 20, 30, 40, 50],
    "shipping_price": [100, 200, 300, 400, 500],
    "subtotal": [150,250,350,450,550],
    "total_price": [105, 210, 315, 420, 525]
}
df= pd.DataFrame(data)

clients_of_interest = [33615, 66037, 46820, 38378, 24741]
filtered_df = df[df["client"].inis(clients_of_interest)]
    
total_qty = {}
total_shipping_price = {}
total_subtotal = {}
total_total_price = {}

for index, row in filtered_df.iterrows():
    client = row["client"]
    qty = row["qty"]
    shipping_price = row["shipping_price"]
    subtotal = row["subtotal"]
    total_price = row["total_price"]

    if client in total_qty:
        total_qty[client] += qty
        total_shipping_price[client] += shipping_price
        total_subtotal[client] += subtotal
        total_total_price[client] += total_price
    else:
        total_qty[client] = qty
        total_shipping_price[client] = shipping_price
        total_subtotal[client] = subtotal
        total_total_price[client] = total_price

summary_data = {
    "client": clients_of_interest
    "total_qty": [total_qty.get(client, 0) for client in clients_of_interest],
    "total_shipping_price": [total_shipping_price.get(client, 0) for client in clients_of_interest],
    "total_subtotal": [total_subtotal.get(client, 0) for client in clients_of_interest],
    "total_total_price": [total_total_price.get(client, 0) for client in clients_of_interest]
}

summary_df = pd.DataFrame(summary_data)
print(summary_df)

In [ ]:
# Format the data and rename the columns to names suitable for presentation.
summary_df = summary_df.rename(columns={
    "Client ID": "Client ID",
    "Total QTY": "Total Units Purchased",
    "Total Shipping Price": "Total Shipping Cost",
    "Total Subtotal": "Total Revenue",
    "Total Total Price": "Total Profit"
})
# Define the money columns. 
money_columns = ["Total Shipping Price", "Total Revenue", "Total Profit"]
df[money_columns] = df[money_columns].astype(float)

# Define a function that converts a dollar amount to millions.
def format_currency(value):
    return '${:,.2f}'.format(value)
for column in money_columns:
    df[column] = df[column].apply(format_currency)

# Apply the currency_format_millions function to only the money columns. 
top_five_clients["total_units_purchased"] = top_five_clients["total_units_purchased"].apply(lambda x: "{:,.0f}".format(x))
top_five_clients["total_shipping_price"] = top_five_clients["total_shipping_price"].apply(format_currency)
top_five_clients["total_revenue"] = top_five_clients["total_revenue"].apply(format_currency)
top_five_clients["total_profit"] = top_five_clients["total_profit"].apply(format_currency)

# Rename the columns to reflect the change in the money format. 
new_column_names = {
    "Total Shipping Price": "Total Shipping Price (USD)",
    "Total Revenue": "Total Revenue (USD)",
    "Total Profit": "Total Profit (USD)"
}

df = df.rename(columns=new_column_names)

print(df)

In [ ]:
# Sort the updated data by "Total Profit (millions)" form highest to lowest and assign the sort to a new DatFrame.
sorted_data = .sort_values(by="Total Profit (millions)", ascending=False)